This is the required imports. We use the requests to request data form the API and we import the pandas and json_normalize to get better vision of the data, which makes it more easy to analyze from

In [130]:
import requests
import pandas as pd
from pandas import json_normalize

In [30]:
# This is the API-token that we got from sportmonks on the "free" subscription.
# This should give us data from the danish superliga and the scottish first league
API_TOKEN = "uaHKOLCbUA35mY7wLI97jgXsmaS0xDohDasg0OwQ26Cfo29xJ3PtYmj9NbVK"

In [31]:
#Lets see if that works by loading the leagues from the free plan

leagues = requests.get("https://api.sportmonks.com/v3/football/leagues?api_token="+API_TOKEN)

leagues = leagues.json()

In [ ]:
#Lets see the results
leagues

In [19]:
# Now I will try to print all the leagues, to see clearly which leagues this data contains

for league in leagues["data"]:
    print(league["id"], league["name"], league["country_id"])

271 Superliga 320
501 Premiership 1161
513 Premiership Play-Offs 1161
1659 Superliga Play-offs 320


In [137]:
# API key
api_key = "uaHKOLCbUA35mY7wLI97jgXsmaS0xDohDasg0OwQ26Cfo29xJ3PtYmj9NbVK"

# API endpoint URL
base_url = "https://api.sportmonks.com/v3/football"
endpoint = "/players"
include = "position;country;city" 
endpoint_url = f"{base_url}{endpoint}?include={include}"

headers = {"Authorization": api_key}

response = requests.get(endpoint_url, headers=headers)

data = response.json()

rows = []
for player in data.get("data", []):
    position = player.get("position") or {}
    country = player.get("country") or {}
    city = player.get("city") or {}
    rows.append([player.get("name"), position.get("name"), country.get("name"), city.get("name")])

df = pd.DataFrame(rows, columns=["Player","Position","Country","City"])
df

,Player,Position,Country,City
0,Daniel Munthe Agger,Coach,Denmark,Hvidovre
1,Liam Miller,Midfielder,Republic of Ireland,None
2,Anthony Stokes,Attacker,Republic of Ireland,None
3,Jermain Defoe,None,England,London
4,Craig Gordon,Goalkeeper,Scotland,None
5,Ross Wallace,Midfielder,Scotland,None
6,Nicklas Bendtner,Attacker,Denmark,None
7,Kolo Habib Touré,Defender,Ivory Coast,None
8,Bernard Mendy,Defender,France,Evreux
9,Daryl Murphy,Attacker,Republic of Ireland,None
